In [1]:
from sumo_python.helpers.load_experiment_metadata import load_experiment_config

input_path = "/Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/"

config, sim_setup = load_experiment_config(config=input_path + 'config.json',
                        sim_setup=input_path + 'simulation_setups.json',)

In [3]:
import numpy as np
import pandas as pd

from sumo_python.helpers.generate_od_xml import generate_od_xml
from sumo_python.simulator.run_sumo_and_parse_output import run_sumo_and_parse_output
from sumo_python.simulator.ground_truth import run_ground_truth_and_parse
from sumo_python.helpers.nrmse_counts import compute_nrmse_counts

df_gt_data = run_ground_truth_and_parse(config, sim_setup)


od2trips --no-step-log  --spread.uniform --taz-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.taz.xml --tazrelation-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.gt_od.xml -o /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/gt_od_trips.trips.xml 
Success.
sumo --output-prefix gt_ --ignore-route-errors=true --net-file=/Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.net.xml --routes=/Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/gt_od_trips.trips.xml -b 54000 -e 57600 --additional-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.additional.xml --duration-log.statistics --xml-validation never --vehroutes /Users/rodrse/Documents/DemandCalibration/notebooks

In [4]:
def objective(x: np.array) -> pd.DataFrame:
    """Compute objective variable. This is the NRMSE loss between ground-truth output
    and the output of the simulation output evaluated at x.

    Args:
        x (np.array): Numpy array of counts to generate TAZ file from.

    Returns:
        pd.Dataframe: Data frame of sampling locations and loss esimate.
    """
    prefix_run = 'iter'    
    generate_od_xml(x, config, sim_setup, prefix_run)
    df_simulated = run_sumo_and_parse_output(config, sim_setup, prefix_run)
    loss_estimate = compute_nrmse_counts(df_gt_data, df_simulated)
    df_output = pd.DataFrame(x.reshape(-1, len(x)),columns = ['x1','x2','x3','x4'])
    df_output['loss_estimate'] = loss_estimate
    
    return df_output

## Vanilla BO with Expected Improvement

In [5]:
# Bayesian optimization
import os
import torch
import pandas as pd

device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
dtype = torch.double

### Declare search space
# dimensionality of input space
d = 4 
bounds = torch.tensor([[400,400,400,400],[1000,1000,1000,1000]], device=device, dtype=dtype) 
bounds


tensor([[ 400.,  400.,  400.,  400.],
        [1000., 1000., 1000., 1000.]], dtype=torch.float64)

In [6]:
### Generate batch of policies using Sobol sequences
from torch.quasirandom import SobolEngine
n0 = 20
dim = 4
sobol = SobolEngine(dim, scramble=True)
x_0 = sobol.draw(n0).to(dtype=dtype)
from botorch.utils.transforms import unnormalize
# map the normalized into the original parameter space
train_X0 = unnormalize(x_0, bounds)

df_0 = [
    objective(x) for x in train_X0.cpu().detach().numpy()
]

df_0 = pd.concat(df_0)
df_0['iter'] = 0


printing /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml
### Runing: iter
od2trips --no-step-log  --spread.uniform --taz-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.taz.xml --tazrelation-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml -o /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_od_trips.trips.xml 
Success.
Loading net-file from '/Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.net.xml' ... done (10ms).
Loading additional-files from '/Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.additional.xml' ... done (4ms).
Loading done.
Simulation version 1.16.0 started with time: 54000.00.


Step #57600.00 (11ms ~= 90.91*RT, ~79181.82UPS, vehicles TOT 2902 ACT 871 BUF 0)          
Simulation ended at time: 57600.00
Reason: The final simulation step has been reached.
Performance: 
 Duration: 22.05s
 Real time factor: 163.288
 UPS: 92367.306209
Vehicles: 
 Inserted: 2902
 Running: 871
 Waiting: 0
 Teleports: 13 (Jam: 4, Wrong Lane: 9)
Statistics (avg of 2031):
 RouteLength: 4398.98
 Speed: 10.93
 Duration: 514.63
 WaitingTime: 102.74
 TimeLoss: 195.71
 DepartDelay: 0.47

DijkstraRouter answered 2902 queries and explored 10.26 edges on average.
DijkstraRouter spent 0.02s answering queries (0.01ms on average).
printing /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml
### Runing: iter
od2trips --no-step-log  --spread.uniform --taz-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.taz.xml --tazrelation-files /Users/rodrse/Documents/DemandCalibr

Step #57600.00 (8ms ~= 125.00*RT, ~104500.00UPS, vehicles TOT 2639 ACT 836 BUF 296)       
Simulation ended at time: 57600.00
Reason: The final simulation step has been reached.
Performance: 
 Duration: 19.70s
 Real time factor: 182.769
 UPS: 105552.672996
Vehicles: 
 Inserted: 2639 (Loaded: 2935)
 Running: 836
 Waiting: 296
 Teleports: 27 (Jam: 14, Wrong Lane: 13)
Statistics (avg of 1803):
 RouteLength: 4370.52
 Speed: 12.36
 Duration: 441.10
 WaitingTime: 84.09
 TimeLoss: 123.81
 DepartDelay: 0.49

DijkstraRouter answered 2936 queries and explored 10.72 edges on average.
DijkstraRouter spent 0.01s answering queries (0.00ms on average).
printing /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml
### Runing: iter
od2trips --no-step-log  --spread.uniform --taz-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.taz.xml --tazrelation-files /Users/rodrse/Doc

Step #57600.00 (9ms ~= 111.11*RT, ~89222.22UPS, vehicles TOT 3045 ACT 803 BUF 410)        
Simulation ended at time: 57600.00
Reason: The final simulation step has been reached.
Performance: 
 Duration: 23.53s
 Real time factor: 152.996
 UPS: 98766.935827
Vehicles: 
 Inserted: 3045 (Loaded: 3455)
 Running: 803
 Waiting: 410
 Teleports: 13 (Jam: 8, Wrong Lane: 5)
Statistics (avg of 2242):
 RouteLength: 4328.55
 Speed: 11.15
 Duration: 508.83
 WaitingTime: 107.87
 TimeLoss: 194.71
 DepartDelay: 0.49

DijkstraRouter answered 3595 queries and explored 9.94 edges on average.
DijkstraRouter spent 0.01s answering queries (0.00ms on average).
printing /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml
### Runing: iter
od2trips --no-step-log  --spread.uniform --taz-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.taz.xml --tazrelation-files /Users/rodrse/Docume

Step #57600.00 (7ms ~= 142.86*RT, ~131285.71UPS, vehicles TOT 2905 ACT 919 BUF 0)         
Simulation ended at time: 57600.00
Reason: The final simulation step has been reached.
Performance: 
 Duration: 20.41s
 Real time factor: 176.384
 UPS: 102993.679569
Vehicles: 
 Inserted: 2905
 Running: 919
 Waiting: 0
 Teleports: 2 (Yield: 2)
Statistics (avg of 1986):
 RouteLength: 4361.94
 Speed: 10.40
 Duration: 596.82
 WaitingTime: 114.95
 TimeLoss: 281.52
 DepartDelay: 0.50

DijkstraRouter answered 2905 queries and explored 10.15 edges on average.
DijkstraRouter spent 0.01s answering queries (0.00ms on average).
printing /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml
### Runing: iter
od2trips --no-step-log  --spread.uniform --taz-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.taz.xml --tazrelation-files /Users/rodrse/Documents/DemandCalibration/noteboo

...

Step #57600.00 (8ms ~= 125.00*RT, ~99750.00UPS, vehicles TOT 2677 ACT 798 BUF 0)          
Simulation ended at time: 57600.00
Reason: The final simulation step has been reached.
Performance: 
 Duration: 16.76s
 Real time factor: 214.784
 UPS: 103394.904839
Vehicles: 
 Inserted: 2677
 Running: 798
 Waiting: 0
 Teleports: 1 (Yield: 1)
Statistics (avg of 1879):
 RouteLength: 4249.88
 Speed: 11.28
 Duration: 533.16
 WaitingTime: 144.32
 TimeLoss: 225.19
 DepartDelay: 0.49

DijkstraRouter answered 2677 queries and explored 9.61 edges on average.
DijkstraRouter spent 0.01s answering queries (0.00ms on average).
printing /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml
### Runing: iter
od2trips --no-step-log  --spread.uniform --taz-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.taz.xml --tazrelation-files /Users/rodrse/Documents/DemandCalibration/notebook

Step #57600.00 (9ms ~= 111.11*RT, ~99777.78UPS, vehicles TOT 3143 ACT 898 BUF 147)        
Simulation ended at time: 57600.00
Reason: The final simulation step has been reached.
Performance: 
 Duration: 24.18s
 Real time factor: 148.871
 UPS: 92649.987594
Vehicles: 
 Inserted: 3143 (Loaded: 3290)
 Running: 898
 Waiting: 147
 Teleports: 14 (Jam: 8, Wrong Lane: 6)
Statistics (avg of 2245):
 RouteLength: 4382.84
 Speed: 11.39
 Duration: 477.04
 WaitingTime: 82.13
 TimeLoss: 159.36
 DepartDelay: 0.49

DijkstraRouter answered 3335 queries and explored 10.23 edges on average.
DijkstraRouter spent 0.02s answering queries (0.01ms on average).
printing /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml
### Runing: iter
od2trips --no-step-log  --spread.uniform --taz-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.taz.xml --tazrelation-files /Users/rodrse/Docume

Step #57600.00 (10ms ~= 100.00*RT, ~77100.00UPS, vehicles TOT 2767 ACT 771 BUF 16)        
Simulation ended at time: 57600.00
Reason: The final simulation step has been reached.
Performance: 
 Duration: 20.21s
 Real time factor: 178.147
 UPS: 90115.449327
Vehicles: 
 Inserted: 2767 (Loaded: 2783)
 Running: 771
 Waiting: 16
 Teleports: 11 (Jam: 5, Wrong Lane: 6)
Statistics (avg of 1996):
 RouteLength: 4328.57
 Speed: 11.59
 Duration: 476.45
 WaitingTime: 100.53
 TimeLoss: 161.93
 DepartDelay: 0.49

DijkstraRouter answered 2783 queries and explored 10.03 edges on average.
DijkstraRouter spent 0.02s answering queries (0.01ms on average).
printing /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml
### Runing: iter
od2trips --no-step-log  --spread.uniform --taz-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.taz.xml --tazrelation-files /Users/rodrse/Docume

printing /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml
### Runing: iter
od2trips --no-step-log  --spread.uniform --taz-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.taz.xml --tazrelation-files /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_quickstart.current_od.xml -o /Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/iter_od_trips.trips.xml 
Success.
Loading net-file from '/Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.net.xml' ... done (8ms).
Loading additional-files from '/Users/rodrse/Documents/DemandCalibration/notebooks/od_calibration_sumo_files/quickstart/data/quickstart.additional.xml' ... done (2ms).
Loading done.
Simulation version 1.16.0 started with time: 54000.00.


Step #57600.00 (10ms ~= 100.00*RT, ~89500.00UPS, vehicles TOT 3039 ACT 895 BUF 311)       
Simulation ended at time: 57600.00
Reason: The final simulation step has been reached.
Performance: 
 Duration: 25.46s
 Real time factor: 141.42
 UPS: 89598.130107
Vehicles: 
 Inserted: 3039 (Loaded: 3350)
 Running: 895
 Waiting: 311
 Teleports: 25 (Jam: 14, Wrong Lane: 11)
Statistics (avg of 2144):
 RouteLength: 4440.82
 Speed: 12.28
 Duration: 430.30
 WaitingTime: 71.54
 TimeLoss: 108.64
 DepartDelay: 0.46

DijkstraRouter answered 3392 queries and explored 10.72 edges on average.
DijkstraRouter spent 0.03s answering queries (0.01ms on average).


In [ ]:
### GP model surrogate
import torch
import matplotlib.pyplot as plt

from botorch.models.transforms import Normalize, Standardize

from botorch.utils.transforms import normalize

from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_mll


train_X = torch.from_numpy(df_0[["x1","x2","x3","x4"]].values)
train_X_norm = normalize(train_X,bounds)

train_Y = -torch.from_numpy(df_0[["loss_estimate"]].values)


def initialize_gp_model(train_X,train_Y):
            
    gp_model = SingleTaskGP(
        train_X=train_X,
        train_Y=train_Y,
        outcome_transform=Standardize(m=1)
    )

    gp_mll = ExactMarginalLogLikelihood(gp_model.likelihood, gp_model)
    
    return gp_model, gp_mll


gp_model, gp_mll = initialize_gp_model(train_X_norm,train_Y)

# Fit model
fit_gpytorch_mll(gp_mll)


In [ ]:

### Acquisition Function: q-EI
# Acquisition function
from botorch.acquisition import qExpectedImprovement
from botorch.sampling.stochastic_samplers import StochasticSampler

sampler = StochasticSampler(sample_shape=torch.Size([128]))
qEI = qExpectedImprovement(gp_model, best_f=max(train_Y), sampler=sampler)

from botorch.utils.transforms import unnormalize
from botorch.optim import optimize_acqf
    
def optimize_acqf_and_get_observation(acq_func,bounds):
    """Optimizes the acquisition function, and returns a new candidate."""
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=torch.tensor([[0.0] * dim, [1.0] * dim], device=device, dtype=dtype),
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 200},
    )

    # observe new values 
    new_x = candidates.detach()
    
    return unnormalize(new_x, bounds)


In [ ]:

BATCH_SIZE = 1 
NUM_RESTARTS = 5 
RAW_SAMPLES = 32 

new_x = optimize_acqf_and_get_observation(
    acq_func=qEI,
    bounds=bounds
)

new_x


In [ ]:

### Run loop
NITER = 175
output = [df_0]
best_value = []

for i in range(NITER):
    
    print(f"####### {i+1} ########")
    
    df_ = df_0 if i == 0 else df_i
    
    n_obs = df_.shape[0]
    print(f"##### n={n_obs} #####")
    
    # Training data
    train_X = torch.from_numpy(df_[["x1","x2","x3","x4"]].values)    
    train_X_norm = normalize(train_X,bounds)
    
    train_Y = -torch.from_numpy(df_[["loss_estimate"]].values) # Take negative
    
    # best value so far
    best_y = train_Y.max()
    best_value.append(best_y)
    print(f"##### best_value={best_y} #####")

    # Declare model with newest data
    gp_model, gp_mll = initialize_gp_model(train_X_norm,train_Y)

    # Fit model
    fit_gpytorch_mll(gp_mll)

    # Construct acquistion function 
    sampler = StochasticSampler(sample_shape=torch.Size([128]))
    qEI = qExpectedImprovement(gp_model, best_f=best_y, sampler=sampler)
    
    # Maximize acquisition function to get next observation
    new_x = optimize_acqf_and_get_observation(acq_func=qEI,bounds=bounds)
    
    # Query SUMO on new sampling location
    df_simulation = objective(new_x.cpu().detach().numpy().reshape(-1))
    df_simulation['iter'] = i+1
    
    # Update history of training data
    output.append(df_simulation)
    
    df_i = pd.concat(output)

df = pd.concat(output)


In [ ]:

df_plot = df.query('iter>0')
import numpy as np
import matplotlib.pyplot as plt

x = df_plot['iter']
y = df_plot['loss_estimate']

plt.plot(x, best_value)
#plt.legend(title='Parameter where:')
plt.show()
df.query('loss_estimate<0.010')